In [21]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set(font="IPAexGothic",style="white")
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import LinearRegression as LR
from sklearn.ensemble import RandomForestRegressor as RF
import lightgbm as lgb

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submit.csv",header=None)
print("Data Shapes")
print("Train:",train.shape, "Test:",test.shape, "Sample:",sample.shape)

Data Shapes
Train: (1200, 34) Test: (800, 33) Sample: (800, 2)


In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(font="IPAexGothic",style="white")

In [4]:
test.isnull().sum()

id                          0
Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int

In [7]:
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,Travel_Rarely,450.941476,Research & Development,7.601074,3,Medical,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,Travel_Rarely,730.235896,Research & Development,26.739489,3,Technical Degree,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,Travel_Rarely,1082.560066,Human Resources,7.374739,3,Life Sciences,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,Travel_Rarely,706.247579,Research & Development,14.791373,1,Medical,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,Travel_Rarely,500.610860,Research & Development,2.146966,4,Medical,981,2,...,80,1,8,3,3,1,0,0,0,0


In [8]:
train.describe()

,id,Age,DailyRate,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,...,1200.0,1200.000000,1200.000000,1200.000000,1200.00000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,1013.075833,32.701667,750.754157,10.218387,2.853333,1077.855000,2.655833,66.261667,2.716667,2.125833,...,80.0,0.863333,10.631667,2.755000,2.70000,6.351667,4.249167,2.163333,3.780000,0.177500
std,575.726036,8.282199,415.420917,8.134144,1.060390,580.185495,1.139502,20.257801,0.675847,1.185821,...,0.0,0.874810,7.721335,1.256167,0.70267,5.120532,3.688507,3.312219,3.314225,0.382251
min,0.000000,17.000000,59.231581,-0.023999,1.000000,12.000000,1.000000,30.000000,1.000000,1.000000,...,80.0,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,508.750000,26.000000,403.946043,2.688173,2.000000,699.000000,2.000000,48.000000,2.000000,1.000000,...,80.0,0.000000,5.000000,2.000000,2.00000,2.000000,2.000000,0.000000,2.000000,0.000000
50%,1018.000000,34.000000,736.873463,9.345924,3.000000,1059.000000,3.000000,67.000000,3.000000,2.000000,...,80.0,1.000000,9.000000,3.000000,3.00000,5.000000,3.000000,1.000000,3.000000,0.000000
75%,1519.250000,37.000000,1092.443495,15.231438,4.000000,1591.000000,4.000000,84.000000,3.000000,3.000000,...,80.0,1.000000,15.000000,3.000000,3.00000,9.000000,7.000000,2.000000,7.000000,0.000000
max,1998.000000,56.000000,1484.979305,29.890208,5.000000,2060.000000,4.000000,100.000000,4.000000,5.000000,...,80.0,3.000000,36.000000,6.000000,4.00000,32.000000,17.000000,15.000000,17.000000,1.000000


In [12]:
train.Attrition

0       0
1       0
2       0
3       1
4       0
       ..
1195    0
1196    0
1197    0
1198    0
1199    1
Name: Attrition, Length: 1200, dtype: int64

# カテゴリ変数をラベルエンコーディング

In [29]:
train.describe(include="O")

,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
count,1200,1200,1200,1200,1200,1200,1200,1200
unique,3,3,6,2,9,3,1,2
top,Travel_Rarely,Research & Development,Life Sciences,Male,Research Scientist,Married,Y,No
freq,808,773,501,713,252,578,1200,882


In [68]:
print(f'BusinessTravel:{pd.unique(train["BusinessTravel"])}')
print(f'Department:{pd.unique(train["Department"])}')
print(f'EducationField:{pd.unique(train["EducationField"])}')
print(f'Gender:{pd.unique(train["Gender"])}')
print(f'JobRole:{pd.unique(train["JobRole"])}')
print(f'MaritalStatus:{pd.unique(train["MaritalStatus"])}')
print(f'Over18:{pd.unique(train["Over18"])}')
print(f'OverTime:{pd.unique(train["OverTime"])}')

BusinessTravel:['Travel_Rarely' 'Non-Travel' 'Travel_Frequently']
Department:['Research & Development' 'Human Resources' 'Sales']
EducationField:['Medical' 'Technical Degree' 'Life Sciences' 'Marketing' 'Other'
 'Human Resources']
Gender:['Male' 'Female']
JobRole:['Laboratory Technician' 'Research Scientist' 'Human Resources'
 'Manufacturing Director' 'Sales Executive' 'Sales Representative'
 'Healthcare Representative' 'Research Director' 'Manager']
MaritalStatus:['Single' 'Divorced' 'Married']
Over18:['Y']
OverTime:['No' 'Yes']


# BusinessTravel→オンホットエンコーディング

In [71]:
train.groupby("BusinessTravel").count()["Attrition"]

BusinessTravel
Non-Travel           129
Travel_Frequently    263
Travel_Rarely        808
Name: Attrition, dtype: int64

# BusinessTravelがAttritionと関係ありそうか確認

In [79]:
#あるカラムの条件を満たすものだけを抽出
train[train["BusinessTravel"] == "Travel_Rarely"].groupby("Attrition").count()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Attrition,,,,,,,,,,,,,,,,,,,,,
0,651,651,651,651,651,651,651,651,651,651,...,651,651,651,651,651,651,651,651,651,651
1,157,157,157,157,157,157,157,157,157,157,...,157,157,157,157,157,157,157,157,157,157


In [80]:
train[train["BusinessTravel"] == "Travel_Frequently"].groupby("Attrition").count()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Attrition,,,,,,,,,,,,,,,,,,,,,
0,212,212,212,212,212,212,212,212,212,212,...,212,212,212,212,212,212,212,212,212,212
1,51,51,51,51,51,51,51,51,51,51,...,51,51,51,51,51,51,51,51,51,51


In [81]:
train[train["BusinessTravel"] == "Non-Travel"].groupby("Attrition").count()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Attrition,,,,,,,,,,,,,,,,,,,,,
0,124,124,124,124,124,124,124,124,124,124,...,124,124,124,124,124,124,124,124,124,124
1,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


# get_dummies

In [83]:
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,Travel_Rarely,450.941476,Research & Development,7.601074,3,Medical,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,Travel_Rarely,730.235896,Research & Development,26.739489,3,Technical Degree,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,Travel_Rarely,1082.560066,Human Resources,7.374739,3,Life Sciences,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,Travel_Rarely,706.247579,Research & Development,14.791373,1,Medical,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,Travel_Rarely,500.610860,Research & Development,2.146966,4,Medical,981,2,...,80,1,8,3,3,1,0,0,0,0


In [85]:
dummy_train = pd.get_dummies(train)
dummy_train

,id,Age,DailyRate,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0,26,450.941476,7.601074,3,1291,4,43,2,1,...,0,0,0,0,0,0,1,1,1,0
1,3,47,730.235896,26.739489,3,1587,2,98,2,1,...,0,1,0,0,1,0,0,1,0,1
2,7,26,1082.560066,7.374739,3,1591,2,84,2,1,...,0,0,0,0,0,1,0,1,1,0
3,10,46,706.247579,14.791373,1,1572,1,79,2,1,...,0,0,0,0,1,0,0,1,0,1
4,11,25,500.610860,2.146966,4,981,2,98,3,2,...,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1988,49,969.251891,13.243579,2,1587,4,67,3,5,...,1,0,0,0,0,1,0,1,1,0
1196,1994,37,437.940367,2.339438,1,137,4,82,3,2,...,0,0,0,0,0,1,0,1,0,1
1197,1996,40,978.883360,10.214979,3,1587,3,40,2,3,...,0,0,0,0,1,0,0,1,1,0
1198,1997,37,170.494984,2.639879,2,120,1,93,4,2,...,0,0,1,0,1,0,0,1,1,0


# dummy_trainでランダムフォレストやってみる

In [86]:
#trainデータから説明変数と目的変数に分ける
dm_train_y = dummy_train.Attrition
dm_train_x = dummy_train.drop("Attrition",axis=1)
print(dm_train_y.shape,dm_train_x.shape)

(1200,) (1200, 54)


In [87]:
from sklearn.model_selection import train_test_split
# 学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(dm_train_x,dm_train_y,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=y)

In [90]:
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
clf = RandomForestClassifier(random_state=1234)
clf.fit(X_train, y_train)
print("train_score=", clf.score(X_train, y_train))
print("test_score=", clf.score(X_test, y_test))

train_score= 1.0
test_score= 0.8125


# testデータに適用

In [91]:
test.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1,34,Travel_Frequently,358.039056,Research & Development,9.926847,3,Life Sciences,843,1,...,3,80,1,11,2,3,7,7,2,7
1,2,37,Travel_Rarely,749.376719,Research & Development,10.391959,3,Medical,1361,1,...,4,80,0,5,3,2,1,3,1,0
2,4,34,Travel_Frequently,958.484177,Research & Development,3.297025,3,Life Sciences,975,2,...,3,80,1,21,2,2,4,3,1,4
3,5,34,Travel_Rarely,79.174592,Research & Development,2.430032,4,Medical,1591,3,...,4,80,3,9,3,3,20,6,1,4
4,6,29,Travel_Rarely,943.191218,Research & Development,10.017149,2,Life Sciences,1803,3,...,3,80,1,25,2,4,1,7,0,0


In [99]:
dummy_test = pd.get_dummies(test)
pred = clf.predict(dummy_test)
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,

In [100]:
sample

,0,1
0,1,0
1,2,0
2,4,0
3,5,0
4,6,0
...,...,...
795,1991,0
796,1992,0
797,1993,0
798,1995,0


In [101]:
sample[1] = pred
sample

,0,1
0,1,0
1,2,0
2,4,0
3,5,0
4,6,0
...,...,...
795,1991,0
796,1992,0
797,1993,0
798,1995,0


In [102]:
# dfをcsv出力　indexをTrueにしたら0からインデックスが降られる、headerはカラム名が一番上に追加される
sample.to_csv("submit01.csv", index = False, header=False)

## ここから下は完成してるセル
# 　テスト作業は上のセルで行うようにする

## trainデータから説明変数と目的変数に分ける

In [55]:
#trainデータから説明変数と目的変数に分ける
train_y = train.Attrition
train_x = train.drop("Attrition",axis=1)
print(train_y.shape,train_x.shape)

(1200,) (1200, 33)


# 学習データとテストデータに分ける

In [45]:
from sklearn.model_selection import train_test_split
# 学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(train_x,train_y,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=ty)

#　ランダムフォレストモデル作成

In [51]:
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
clf = RandomForestClassifier(random_state=1234)
clf.fit(X_train, y_train)
print("score=", clf.score(X_test, y_test))

ValueError: could not convert string to float: 'Travel_Rarely'

In [53]:
train

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,Travel_Rarely,450.941476,Research & Development,7.601074,3,Medical,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,Travel_Rarely,730.235896,Research & Development,26.739489,3,Technical Degree,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,Travel_Rarely,1082.560066,Human Resources,7.374739,3,Life Sciences,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,Travel_Rarely,706.247579,Research & Development,14.791373,1,Medical,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,Travel_Rarely,500.610860,Research & Development,2.146966,4,Medical,981,2,...,80,1,8,3,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1988,49,Travel_Rarely,969.251891,Research & Development,13.243579,2,Medical,1587,4,...,80,0,33,3,2,3,2,1,2,0
1196,1994,37,Travel_Frequently,437.940367,Research & Development,2.339438,1,Other,137,4,...,80,0,6,5,2,9,2,0,8,0
1197,1996,40,Non-Travel,978.883360,Human Resources,10.214979,3,Life Sciences,1587,3,...,80,1,19,2,3,14,9,9,9,0
1198,1997,37,Travel_Frequently,170.494984,Sales,2.639879,2,Marketing,120,1,...,80,2,6,5,3,3,2,0,3,0
